In [1]:
# 데이터프레임
import pandas as pd
# 웹 서버에 요청을 보내고 응답 메시지 받는 기능
import requests
# 크롤링 기능
from bs4 import BeautifulSoup as bs
# 웹 브라우저 제어
from selenium import webdriver
# 브라우저의 태그 검색 find.elements or find.element
from selenium.webdriver.common.by import By
# 키보드의 특수 이벤트 (ENTER,DELETE, CLEAR)
from selenium.webdriver.common.keys import Keys

- 네이버 증권
    - 종목코드를 이용해서 검색
        - url의 규칙 확인
    - 뉴스 탭에 더보기 버튼을 선택
        - 클릭 이벤트(페이지 이동)
        - url의 규칙 확인
    - 뉴스 리스트에 있는 하이퍼링크 주소를 모두 리스트의 형태로 저장 ['href']
        - requests를 사용해서 해당 주소로 요청을 보내고
        - 정보를 확인하기
        - 기사의 내용이 모두 응답 메시지가 존재한다면
        - 제목, 본문 내용으로 1차원 리스트 생성
        - 모든 기사 내용을 리스트 추가 ( 2차원 리스트 )
        - pandas 이용 DF 생성
        - csv 파일 저장, 파일 이름이 겹치지 않도록 날짜, 시간으로 저장 to_csv ('경로',')

In [2]:
url = 'https://finance.naver.com/item/main.naver?code='

code = '012450' #한화에어로스페이스

In [3]:
# 브라우져 오픈, 특정 주소로 요청 보내기
driver = webdriver.Chrome()
driver.get(url+code)

In [4]:
# driver에서 html코드를 불러와서
# BeautifulSoup를 이용해 데이터 파싱
soup = bs(driver.page_source, 'html.parser')

In [5]:
len(driver.page_source)

232285

In [6]:
div_data = soup.find('div',attrs={
    'class' : 'news_section'
})

In [7]:
len(
    div_data
)

8

In [8]:
li_list = div_data.find_all('li')

In [9]:
li_list

[<li>
 <span class="txt">
 <a href="/item/news_read.naver?article_id=0004460830&amp;office_id=011&amp;code=012450&amp;sm=entity_id.basic" onclick="clickcr(this, 'dle.1', '', '1', event);">현대로템, 유럽 방위비 증액 기대감 타고 연일 최...</a>
 </span>
 <em> 03/13</em>
 </li>,
 <li>
 <span class="txt">
 <a href="/item/news_read.naver?article_id=0000915712&amp;office_id=031&amp;code=012450&amp;sm=entity_id.basic" onclick="clickcr(this, 'dle.1', '', '2', event);">한화에어로, 선박용 수소연료전지 ...</a>
 <a class="link_relation" href="/item/news.naver?code=012450&amp;clusterId=0310000915712" onclick="clickcr(this, 'dle.11', '', '', event);"><span class="bar"></span>관련 <em>11</em>건</a>
 </span>
 <em> 03/13</em>
 </li>,
 <li>
 <span class="txt">
 <a href="/item/news_read.naver?article_id=0000099084&amp;office_id=586&amp;code=012450&amp;sm=entity_id.basic" onclick="clickcr(this, 'dle.1', '', '3', event);">[단독] "쉴드AI 투자자는 한화에어...</a>
 <a class="link_relation" href="/item/news.naver?code=012450&amp;clusterId=5860000099084" oncl

In [10]:
len(li_list)

10

In [11]:
# li_list 각 원소들의 첫번째 a태그를 선택해서 
# href의 속성 값을 이용해 새로운 리스트 생성
# 뉴스 url만 추출
href_list = []
for li_data in li_list:
    data = li_data.find('a')['href']
    href_list.append(data)
href_list

['/item/news_read.naver?article_id=0004460830&office_id=011&code=012450&sm=entity_id.basic',
 '/item/news_read.naver?article_id=0000915712&office_id=031&code=012450&sm=entity_id.basic',
 '/item/news_read.naver?article_id=0000099084&office_id=586&code=012450&sm=entity_id.basic',
 '/item/news_read.naver?article_id=0005165198&office_id=008&code=012450&sm=entity_id.basic',
 '/item/news_read.naver?article_id=0002932499&office_id=119&code=012450&sm=entity_id.basic',
 '/item/news_read.naver?article_id=0000074275&office_id=243&code=012450&sm=entity_id.basic',
 '/item/news_read.naver?article_id=0008127271&office_id=421&code=012450&sm=entity_id.basic',
 '/item/news_read.naver?article_id=0002932472&office_id=119&code=012450&sm=entity_id.basic',
 '/item/news_read.naver?article_id=0005320445&office_id=014&code=012450&sm=entity_id.basic',
 '/item/news_read.naver?article_id=0000429454&office_id=374&code=012450&sm=entity_id.basic']

In [12]:
base_url = 'https://finance.naver.com'

In [13]:
href_list[0]

'/item/news_read.naver?article_id=0004460830&office_id=011&code=012450&sm=entity_id.basic'

In [14]:
res = requests.get(base_url+href_list[0])

In [15]:
print(res.text)

<SCRIPT>top.location.href='https://n.news.naver.com/mnews/article/011/0004460830';</SCRIPT>



In [16]:
# selenium을 이용해서 base_url + href[0]에 요청
driver.get(base_url + href_list[0])

In [17]:
# html 문서를 bs를 이용해서 데이터 파싱
news_soup = bs(driver.page_source,'html.parser')

In [18]:
# h2 태그 중 id가 'title_area'인 태그를 선택하고 텍스트 추출
news_title = news_soup.find('h2',attrs={
    'id' : 'title_area'
}).get_text()
news_title

'현대로템, 유럽 방위비 증액 기대감 타고 연일 최고가[줍줍 리포트]'

In [19]:
# div 태그 중에 id 가 'newsct_article'인 태그를 선택하고 텍스트 추출
news_content = news_soup.find('div',attrs = {
    'id' : 'newsct_article'
}).get_text().replace('\n','')
news_content

'한화에어로 6%, 한국항공우주도 4% 상승 중현대로템 K2 전차. 사진 제공=현대로템[서울경제] 현대로템(064350)이 유럽의 방위비 증액 기대감을 타고 연일 최고가를 경신하고 있다.13일 한국거래소에 따르면 오전 10시 40분 현재 현대로템은 전일 대비 8400원(8.91%) 오른 10만 2700원에 거래되고 있다. 현대로템은 전날에도 장중 한때 11%대까지 오르며 최고가를 경신했다. 이밖에 한화에어로스페이스(012450)도 6.17%대 오르고 있고 항공항공우주(4.51%), 엠앤씨솔루션(484870)(6.75%), 풍산(103140)(3.09%) 등도 동반 오름세다.이날 방산주들이 일제히 오른 것은 도널드 트럼프 미국 대통령 당선 이후 세계 각국의 방위비 증액 기조의 수혜가 이어질 것이라는 기대감이 반영된 결과로 풀이된다. 특히 최근 유럽 국가들이 국방력 강화를 적극 추진하면서, 폴란드를 필두로 유럽에 K2 전차를 수출하고 있는 현대로템은 더욱 가파르게 오르고 있다.최근 한국투자증권은 현대로템 목표주가를 9만 2000원에서 11만 8000원으로 상향 조정했고, KB증권도 8만 2500원에서 12만 원으로 올렸다.장남현 한국투자증권 연구원은 현대로템에 대해 “미국의 자국 우선주의 기조가 짙어지면서 유럽 국가들이 자체 국방 강화를 위한 방위비 증액에 나서고 있다”며 “현대로템은 비유럽 지역에서 경쟁 강도가 낮아지고, 유럽 내 수요 증대에 따른 시장이 확대되는 두 가지 수혜를 모두 누릴 수 있을 것”이라고 밝혔다. 장 연구원은 또 K2 전차 변속기 국산화가 완료되면서 올해를 기점으로 중동 시장 마케팅이 본격화할 것으로 봤다. 사우디아라비아와 아랍에미레이트(UAE)의 교체 수요를 고려한 중동 전차 수출 시장 규모는 약 18조원 이상으로, 경쟁 강도가 완화한 상황에서 시장에 진입할 수 있는 기회가 열릴 것으로 예상했다.정동익 KB증권 연구원은 “K2 전차의 폴란드 2차 계약이 4월 중에 체결될 것이라는 보도가 이어지고 있다”며 “이번 계약은 다양한 옵션에 물가 

In [20]:
news_data = []
news_data.append(news_title)
news_data.append(news_content)
news_data

['현대로템, 유럽 방위비 증액 기대감 타고 연일 최고가[줍줍 리포트]',
 '한화에어로 6%, 한국항공우주도 4% 상승 중현대로템 K2 전차. 사진 제공=현대로템[서울경제] 현대로템(064350)이 유럽의 방위비 증액 기대감을 타고 연일 최고가를 경신하고 있다.13일 한국거래소에 따르면 오전 10시 40분 현재 현대로템은 전일 대비 8400원(8.91%) 오른 10만 2700원에 거래되고 있다. 현대로템은 전날에도 장중 한때 11%대까지 오르며 최고가를 경신했다. 이밖에 한화에어로스페이스(012450)도 6.17%대 오르고 있고 항공항공우주(4.51%), 엠앤씨솔루션(484870)(6.75%), 풍산(103140)(3.09%) 등도 동반 오름세다.이날 방산주들이 일제히 오른 것은 도널드 트럼프 미국 대통령 당선 이후 세계 각국의 방위비 증액 기조의 수혜가 이어질 것이라는 기대감이 반영된 결과로 풀이된다. 특히 최근 유럽 국가들이 국방력 강화를 적극 추진하면서, 폴란드를 필두로 유럽에 K2 전차를 수출하고 있는 현대로템은 더욱 가파르게 오르고 있다.최근 한국투자증권은 현대로템 목표주가를 9만 2000원에서 11만 8000원으로 상향 조정했고, KB증권도 8만 2500원에서 12만 원으로 올렸다.장남현 한국투자증권 연구원은 현대로템에 대해 “미국의 자국 우선주의 기조가 짙어지면서 유럽 국가들이 자체 국방 강화를 위한 방위비 증액에 나서고 있다”며 “현대로템은 비유럽 지역에서 경쟁 강도가 낮아지고, 유럽 내 수요 증대에 따른 시장이 확대되는 두 가지 수혜를 모두 누릴 수 있을 것”이라고 밝혔다. 장 연구원은 또 K2 전차 변속기 국산화가 완료되면서 올해를 기점으로 중동 시장 마케팅이 본격화할 것으로 봤다. 사우디아라비아와 아랍에미레이트(UAE)의 교체 수요를 고려한 중동 전차 수출 시장 규모는 약 18조원 이상으로, 경쟁 강도가 완화한 상황에서 시장에 진입할 수 있는 기회가 열릴 것으로 예상했다.정동익 KB증권 연구원은 “K2 전차의 폴란드 2차 계약이 4월 중에

In [21]:
# news_dict = {}
# news_dict = ['title'] = news_title
# news_dict = ['content'] = news_content

In [22]:
news_dict = {
    'title' : news_title,
    'content' : news_content
}
news_dict

{'title': '현대로템, 유럽 방위비 증액 기대감 타고 연일 최고가[줍줍 리포트]',
 'content': '한화에어로 6%, 한국항공우주도 4% 상승 중현대로템 K2 전차. 사진 제공=현대로템[서울경제] 현대로템(064350)이 유럽의 방위비 증액 기대감을 타고 연일 최고가를 경신하고 있다.13일 한국거래소에 따르면 오전 10시 40분 현재 현대로템은 전일 대비 8400원(8.91%) 오른 10만 2700원에 거래되고 있다. 현대로템은 전날에도 장중 한때 11%대까지 오르며 최고가를 경신했다. 이밖에 한화에어로스페이스(012450)도 6.17%대 오르고 있고 항공항공우주(4.51%), 엠앤씨솔루션(484870)(6.75%), 풍산(103140)(3.09%) 등도 동반 오름세다.이날 방산주들이 일제히 오른 것은 도널드 트럼프 미국 대통령 당선 이후 세계 각국의 방위비 증액 기조의 수혜가 이어질 것이라는 기대감이 반영된 결과로 풀이된다. 특히 최근 유럽 국가들이 국방력 강화를 적극 추진하면서, 폴란드를 필두로 유럽에 K2 전차를 수출하고 있는 현대로템은 더욱 가파르게 오르고 있다.최근 한국투자증권은 현대로템 목표주가를 9만 2000원에서 11만 8000원으로 상향 조정했고, KB증권도 8만 2500원에서 12만 원으로 올렸다.장남현 한국투자증권 연구원은 현대로템에 대해 “미국의 자국 우선주의 기조가 짙어지면서 유럽 국가들이 자체 국방 강화를 위한 방위비 증액에 나서고 있다”며 “현대로템은 비유럽 지역에서 경쟁 강도가 낮아지고, 유럽 내 수요 증대에 따른 시장이 확대되는 두 가지 수혜를 모두 누릴 수 있을 것”이라고 밝혔다. 장 연구원은 또 K2 전차 변속기 국산화가 완료되면서 올해를 기점으로 중동 시장 마케팅이 본격화할 것으로 봤다. 사우디아라비아와 아랍에미레이트(UAE)의 교체 수요를 고려한 중동 전차 수출 시장 규모는 약 18조원 이상으로, 경쟁 강도가 완화한 상황에서 시장에 진입할 수 있는 기회가 열릴 것으로 예상했다.정동익 KB증권 연구원은 “K2 

In [23]:
# delay를 위한 라이브러리 호출
import time

In [ ]:
# data가 data를 저장할 수 있는 빈 list 생성
result = []
base_url = "https://finance.naver.com"
driver = webdriver.Chrome()

# href_list를 이용해서 반복문 실행
for href in href_list:
    # base_url + href_list[index]를 이용하여 webdriver에 요청
    driver.get(base_url+href)
    #2초 정도의 딜레이
    time.sleep(2)
    # 해당 driver에서 html을 추출
    soup = bs(driver.page_source,'html.parser')
    # h2태그 중 id가 title_area인 태그에서 텍스트 추출, 저장
    news_title = soup.find(
        'h2',
        attrs={
            'id' : 'title_area'
        }
    ).get_text()
    # div태그 중 id가 newsct_article인 태그에서 텍스트 추출, 저장
    news_content = soup.find(
        'div',
        attrs = {
        'id' : 'newsct_article'
        }
    ).get_text().replace('\n','')
    # news_dict 생성하여 h2 태그의 텍스트와 div의 태그의 텍스트를 저장
    news_dict={
        'title' : news_title,
        'content' : news_content
    }
    # result에 news_dict 추가
    result.append(news_dict)

In [28]:
result

[{'title': '현대로템, 유럽 방위비 증액 기대감 타고 연일 최고가[줍줍 리포트]',
  'content': '한화에어로 6%, 한국항공우주도 4% 상승 중현대로템 K2 전차. 사진 제공=현대로템[서울경제] 현대로템(064350)이 유럽의 방위비 증액 기대감을 타고 연일 최고가를 경신하고 있다.13일 한국거래소에 따르면 오전 10시 40분 현재 현대로템은 전일 대비 8400원(8.91%) 오른 10만 2700원에 거래되고 있다. 현대로템은 전날에도 장중 한때 11%대까지 오르며 최고가를 경신했다. 이밖에 한화에어로스페이스(012450)도 6.17%대 오르고 있고 항공항공우주(4.51%), 엠앤씨솔루션(484870)(6.75%), 풍산(103140)(3.09%) 등도 동반 오름세다.이날 방산주들이 일제히 오른 것은 도널드 트럼프 미국 대통령 당선 이후 세계 각국의 방위비 증액 기조의 수혜가 이어질 것이라는 기대감이 반영된 결과로 풀이된다. 특히 최근 유럽 국가들이 국방력 강화를 적극 추진하면서, 폴란드를 필두로 유럽에 K2 전차를 수출하고 있는 현대로템은 더욱 가파르게 오르고 있다.최근 한국투자증권은 현대로템 목표주가를 9만 2000원에서 11만 8000원으로 상향 조정했고, KB증권도 8만 2500원에서 12만 원으로 올렸다.장남현 한국투자증권 연구원은 현대로템에 대해 “미국의 자국 우선주의 기조가 짙어지면서 유럽 국가들이 자체 국방 강화를 위한 방위비 증액에 나서고 있다”며 “현대로템은 비유럽 지역에서 경쟁 강도가 낮아지고, 유럽 내 수요 증대에 따른 시장이 확대되는 두 가지 수혜를 모두 누릴 수 있을 것”이라고 밝혔다. 장 연구원은 또 K2 전차 변속기 국산화가 완료되면서 올해를 기점으로 중동 시장 마케팅이 본격화할 것으로 봤다. 사우디아라비아와 아랍에미레이트(UAE)의 교체 수요를 고려한 중동 전차 수출 시장 규모는 약 18조원 이상으로, 경쟁 강도가 완화한 상황에서 시장에 진입할 수 있는 기회가 열릴 것으로 예상했다.정동익 KB증권 연구원은 “K